In [1]:
%matplotlib

import math 
import pywt 
import numpy as np
import os
import csv
import fnmatch
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import scipy.ndimage
from PIL import Image

from matplotlib.font_manager import FontProperties

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


In [2]:
RemoveSecondLook = False
StartFromBeginning = False
Beginning = False
End = False

In [3]:
TimeDwellOrig = 800
TimeFixation = 300

In [4]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [5]:
# Create list of list (made of epochs) composed of times when user looked at scratchPad

def UserLookedAtScratchPad(keysLookedAt):
    epoch = list()   
    epochList = list()

    # TODO: reduce the list to only the first and the last elements of the epoch
    
    nRow = -1
    firstScratchPad = 0
    flagStart = 0
    
    
    # Find unique looks at scratchpad, by checking if progress pct is greater than previous one
    for row in keysLookedAt:
        nRow = nRow + 1
        
        if row[1] == 'ScratchPad':
            
            # for consecutive scratchpad looks
            if keysLookedAt[nRow-1][1] == 'ScratchPad':
                if float(row[2]) == 0:
                    progressScratchPad = float(keysLookedAt[nRow-1][2])
                else:
                    progressScratchPad = float(row[2])
    
            # if first time of the look at scratchpad
            else:
                # if not the very first look (means the variable for progress exists)
                if firstScratchPad > 0: 
                    # if the current progress value is less than the previous one, means it is a new look 
                    if progressScratchPad > float(row[2]):
                        # if progress is not 0, then update the progress --> sometimes, it is 0 at the end of the look
                        if float(row[2]) != 0:
                            progressScratchPad = float(row[2])
                            epochList.append(row)
                            
                # if the very first look (means the variable for progress does not exist)    
                else:
                    progressScratchPad = float(row[2])
                    firstScratchPad = 1
                    epochList.append(row)
    
    #print(epochList)

    return epochList

In [6]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [7]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
    
    return nearestTP, nearestTPind

In [8]:
def UserLooksLocationAfterScratchPad(userKeys, scratchPadEpochs):
    
    scratchPad5sTimeEpoch = list()
    scratchPad5sTimeList = list()
    
    
    # create time list of time in userKeys
    userKeysTimeString = [key[0] for key in userKeys]
    userKeysTime = timeConversion(userKeysTimeString)
    
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in userKeys]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
            
    
    # scratchPad time
    scratchPadTimeString = [key[0] for key in scratchPadEpochs]
    scratchPadTime = timeConversion(scratchPadTimeString)
    
    
    if StartFromBeginning:
        
        if Beginning:
        
            # beginning
            timeStart = userKeysTime[0]
            timeStartInd = 0 
            timeAfter5s = timeStart + datetime.timedelta(seconds=10) 
            timeEnd = nearestTimePoint(userKeysTime, timeAfter5s)
            scratchPad5sTimeEpoch.append([timeStart, timeStartInd, timeEnd[0], timeEnd[1]])
        
        if End:
            # end
            timeEndInd = indBeforeQuit
            timeEnd = userKeysTime[indBeforeQuit]
            timeStartBefore5s = timeEnd - datetime.timedelta(seconds=10)
            timeStart = nearestTimePoint(userKeysTime, timeStartBefore5s)
            scratchPad5sTimeEpoch.append([timeStart[0], timeStart[1], timeEnd, timeEndInd])
        
    else:
        # create list of time of 5s after every look at scratchPad
        for key in scratchPadTime:
        
            timeStart = key
            timeStartInd = userKeysTime.index(timeStart)
            timeAfter5s = timeStart + datetime.timedelta(seconds=5) 
        
            #print(timeStart, timeAfter5s)
        
            timeEnd = nearestTimePoint(userKeysTime, timeAfter5s)
            scratchPad5sTimeEpoch.append([timeStart, timeStartInd, timeEnd[0], timeEnd[1]])
        
            #print(timeStart, timeEnd[0])
    
    
    #print(len(scratchPad5sTimeEpoch))
    
    #if len(scratchPad5sTimeEpoch) < 2:
    #    scratchPad5sTimeList = userKeys[scratchPad5sTimeEpoch[0][1] : scratchPad5sTimeEpoch[0][3]]
    #else:
    #    # create the 5s list
    #    for keyInd in range(0, len(scratchPad5sTimeEpoch)):
    #        scratchPad5sTimeList.append(userKeys[scratchPad5sTimeEpoch[keyInd][1] : scratchPad5sTimeEpoch[keyInd][3]])
        
    scratchPad5sTimeList = [userKeys[scratchPad5sTimeEpoch[keyInd][1]:scratchPad5sTimeEpoch[keyInd][3]] for keyInd in range(0, len(scratchPadEpochs))]
    
    
    scratchPad5sReducedList = list()
    
    if RemoveSecondLook:
        print('remove second look')
        # remove a second look of scratchPad continued in the same look
        # start checking from the back of the list, and if a key other than scratchPad or suggestion exists, then list is 
        # stopped there
        listInd = -1
        userKeysRemove = list()
        for userLooksList in scratchPad5sTimeList:
            
            
            # list to store if second looks present in the same list
            indToRemove = 0
        
            listInd = listInd + 1
            # for every look, ensure that the first key reported is the scratchPad
            if userLooksList[0][1] != 'ScratchPad':
                #print(scratchPad5sTimeList[listInd][0])
                scratchPad5sTimeList[listInd].remove(scratchPad5sTimeList[listInd][0])
        
            # create list of index of looks at scratchPad
            scratchPadInd = [rowInd for rowInd in range(0, len(userLooksList)) if userLooksList[rowInd][1] in keyLocation['scratchPadAndSuggestions'] or userLooksList[rowInd][1] in keyLocation['phraseText']]
               
            # create a list with difference in index of looks at scratchPad
            scratchPadIndDiffArray = np.diff(np.asarray(scratchPadInd), 1)
            scratchPadIndDiff = scratchPadIndDiffArray.tolist()
        
            # create list of keys of the scratchPads
            scratchPadKeys = [row for row in userLooksList if row[1] in keyLocation['scratchPadAndSuggestions'] or row[1] in keyLocation['phraseText']]

            # if another look started, remove the rows of that second look
            if len(scratchPadInd) > 1:
                # continue only if scratchPadInd has more than 1 element
            
                for indofInd in range(0, len(scratchPadIndDiff)):
                    ind = scratchPadIndDiff[indofInd]
                    if ind > 2: # to avoid inadvertant saccades to neighbouring keys
                    
                        # store the (index-1) from where it should be removed, in short, till where to save the list
                        indToRemove = indofInd + ind
                    
                        # inadvertant looks to scratchPad need to be removed, i the look had a progress of less than 300ms
                    
                        # check if the indToRemove is of scratchPad, and that the progress is more than 0, otherwise, 
                        # set indToRemove to 0
                        if scratchPad5sTimeList[listInd][indToRemove][1] == 'ScratchPad':
                            if float(scratchPad5sTimeList[listInd][indToRemove][2]) != 0:
                                # if progress is more than 0
                                # check if the previous scratchPad look was continued
                            
                                if float(scratchPad5sTimeList[listInd][scratchPadInd.index(indofInd)][2]) >= float(scratchPad5sTimeList[listInd][indToRemove][2]):
                                    #print('still the same look')
                                    indToRemove = 0
                                else:
                                    #print(len(userLooksList), indToRemove)
                                    break
                                    
                            else:
                                indToRemove = 0
    
                        else:
                            indToRemove = 0
                            continue
                    
                    
                    
            if indToRemove > 0:
                scratchPad5sReducedList.append(scratchPad5sTimeList[listInd][0:indToRemove])
                print('REMOVE', len(scratchPad5sTimeList[listInd][0:indToRemove]))
            else:
                # if indToRemove is still 0, add the whole list 
                scratchPad5sReducedList.append(scratchPad5sTimeList[listInd])
                print('INTACT', len(scratchPad5sTimeList[listInd]))
    else:
        print('keep 5s')
        # keep complete 5 s
        #print(len(scratchPad5sTimeList))
        scratchPad5sReducedList = scratchPad5sTimeList
        
    
        
    return scratchPad5sReducedList

In [15]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
categoryList = dict()
categoryList['DT'] = list()
categoryList['MS'] = list()

firstSubj = 0

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs and 'mm' in root:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
                        
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
            if userKeys is None or gazeLog is None:
                continue
            else:
                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                
                # Compute dwell time
                userKeysWithTime = ComputeDwellTime(userKeys)
                
                # count % of people looking at scratchPad
                
                scratchPadEpochs = UserLookedAtScratchPad(userKeysWithTime)
                
                #print(scratchPadEpochs)
                
                # gaze fixation point on screen
                xL = [float(item4[20]) if 'Invalid' not in item4 else np.nan for item4 in gazeLog]
                yL = [float(item5[21]) if 'Invalid' not in item5 else np.nan for item5 in gazeLog]
                xR = [float(item4[26]) if 'Invalid' not in item4 else np.nan for item4 in gazeLog]
                yR = [float(item5[27]) if 'Invalid' not in item5 else np.nan for item5 in gazeLog]
                
#                 for i,j in enumerate(xL):
#                     print(i, xL[i], xR[i])
                    
                xPt = [(xL[i]+xR[i])/2 if not np.isnan(xL[i]) else 0 for i,j in enumerate(xL) ]
                yPt = [(yL[i]+yR[i])/2 if not np.isnan(yL[i]) else 0 for i,j in enumerate(yL) ]
                
                scratchPad5sList =  UserLooksLocationAfterScratchPad(userKeysWithTime, scratchPadEpochs)

                position2D = np.zeros([1080, 1920])
                for keyInd,v in enumerate(xPt):
                    x = xPt[keyInd]
                    y = yPt[keyInd]
                    if y>1.0 or y<0 or x<0 or x>1.0:
                        continue
                        print('x: ', x, 'y: ', y)
                    #print(int(np.round(y*1200)), int(np.round(x*1920)))
                    
                    position2D[int(np.round(y*1080))-1][int(np.round(x*1920))-1] = 1
                    
                #print(position2D)
                
                cdict = {
                  'red'  :  ( (0.0, 0.25, .25), (0.02, .59, .59), (1., 1., 1.)),
                  'green':  ( (0.0, 0.0, 0.0), (0.02, .45, .45), (1., .97, .97)),
                  'blue' :  ( (0.0, 1.0, 1.0), (0.02, .75, .75), (1., 0.45, 0.45))
                        }

                #cm = m.colors.LinearSegmentedColormap('my_colormap', cdict, 1024)

                im = np.array(Image.open('optikeyKeyboard.png'), dtype=np.uint8)

                # Create figure and axes
                dpi = 80
                
                # Display the image
                #ax.imshow(im, origin='lower')

                b = scipy.ndimage.filters.gaussian_filter(position2D, sigma = (40, 40))
                fig, ax = plt.subplots(figsize=(15,9), dpi = dpi)
                ax.imshow(im)
                ax.imshow(b, vmin=np.min(b), vmax = np.max(b), cmap='hot', alpha = 0.7)
                # Turn off tick labels
                plt.axis('off')
                ax.set_title(subjName)
                
#                 #print(scratchPad5sList)
#                 if StartFromBeginning:
#                     categoryList = AggregateUserLookLocationOneLook(scratchPad5sList, categoryList, keyLocation, subjName)
#                 else:
#                     categoryList = AggregateUserLookLocation(scratchPad5sList, categoryList, keyLocation, subjName)
                
                

mm_MS
keep 5s


In [ ]:
print(xPt)